In [1]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
from torch.nn.modules.loss import *
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
from pathlib import Path
# %matplotlib notebook

/home/drake/anaconda3/envs/.torch/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/drake/anaconda3/envs/.torch/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/drake/anaconda3/envs/.torch/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/drake/anaconda3/envs/.

In [2]:
from Loss.triplet import *
from session import *
from LR_Schedule.cyclical import Cyclical
from LR_Schedule.cos_anneal import CosAnneal
from LR_Schedule.lr_find import lr_find
from callbacks import *
from validation import *
from validation import _AccuracyMeter
import Datasets.ImageData as ImageData
from Transforms.ImageTransforms import *
import util
import Datasets.ModelData as md

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
transform = transforms.Compose([
                                transforms.RandomRotation(9),
                                transforms.RandomResizedCrop(32, scale=(.95, 1.05)),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(), 
                                transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

trainset = torchvision.datasets.CIFAR10(root='/media/drake/MX500/Datasets/cifar-10/train', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

valset = torchvision.datasets.CIFAR10(root='/media/drake/MX500/Datasets/cifar-10/test', train=False,
                                       download=True, transform=transform)
valloader = torch.utils.data.DataLoader(valset, batch_size=32,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

## Baseline

In [ ]:
model_base = nn.Sequential(
     nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
     nn.BatchNorm2d(32),
     nn.ReLU(),
    
     nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
     nn.BatchNorm2d(32),
     nn.ReLU(),
     
     nn.MaxPool2d(kernel_size=2, stride=2),
    
     nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
     nn.BatchNorm2d(64),
     nn.ReLU(),
     
     nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
     nn.BatchNorm2d(64),
     nn.ReLU(),
     
     nn.MaxPool2d(kernel_size=2, stride=2),
    
     nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
     nn.BatchNorm2d(128),
     nn.ReLU(),
     
     nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
     nn.BatchNorm2d(128),
     nn.ReLU(),
     
     nn.MaxPool2d(kernel_size=2, stride=2),
    
     Flatten(),
    
     nn.Linear(4 * 4 * 128, 512),
     nn.Linear(512, 256),
     nn.Linear(256, 10))

sess_base = Session(model_base, nn.CrossEntropyLoss(), optim.AdamW, 1e-3)

In [ ]:
lr_find(sess_base, trainloader, start_lr=1e-4)

In [ ]:
sess_base.set_lr(1e-3)

In [ ]:
validator = Validator(valloader, OneHotAccuracy(), save_best=True, model_dir='./base')
lr_scheduler = CosAnneal(len(trainloader), T_mult=2)
schedule = TrainingSchedule(trainloader, [lr_scheduler, validator])
sess_base.train(schedule, 63)

## Triplet Loss 


In [6]:
class TripletRegularizedCrossEntropyLoss(nn.Module):
    def __init__(self, alpha, margin):     
        super().__init__()
        self.alpha = alpha
        self.margin = margin
        
    def forward(self, x, y):
        loss = F.cross_entropy(x[-1], y)
        triplet = 0
        
        for layer in x[:-1]:
            triplet += batch_hard_triplet_loss(layer.view(layer.size(0), -1), y, self.margin)
            
        # triplet *= min(self.alpha/math.sqrt(loss.item()), 1)
        triplet *= self.alpha
            
        return loss + triplet
      
class CustomOneHotAccuracy(OneHotAccuracy):
    def __init__(self):
        super().__init__()
        self.reset()

    def update(self, output, label):
        super().update(output[-1], label)
        
class SelectiveSequential(nn.Module):
    def __init__(self, to_select, modules_dict):
        super(SelectiveSequential, self).__init__()
        for key, module in modules_dict.items():
            self.add_module(key, module)
        self._to_select = to_select
    
    def forward(self, x):
        list = []
        for name, module in self._modules.items():              
            #if name in self._to_select:
            #    list.append(module(x.data))
            
            x = module(x)      
            
            if name in self._to_select:
                list.append(x)
                
        return list
    

In [7]:
model = SelectiveSequential(
    ['max3', 'fc1', 'fc2', 'out'],
    {'conv32a': nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
     'batch32a': nn.BatchNorm2d(32),
     'act32a': nn.ReLU(),
    
     'conv32b': nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
     'batch32b': nn.BatchNorm2d(32),
     'act32b': nn.ReLU(),
     
     'max1': nn.MaxPool2d(kernel_size=2, stride=2),
    
     'conv64a': nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
     'batch64a': nn.BatchNorm2d(64),
     'act64a': nn.ReLU(),
     
     'conv64b': nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
     'batch64b': nn.BatchNorm2d(64),
     'act64b': nn.ReLU(),
    
     'max2': nn.MaxPool2d(kernel_size=2, stride=2),
    
     'conv128a': nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
     'batch128a': nn.BatchNorm2d(128),
     'act128a': nn.ReLU(),
     
     'conv128b': nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
     'batch128b': nn.BatchNorm2d(128),
     'act128b': nn.ReLU(),
     
     'max3': nn.MaxPool2d(kernel_size=2, stride=2),
    
     'flatten': Flatten(),
    
     'fc1': nn.Linear(4 * 4 * 128, 512),
     #'drop1': nn.Dropout(.05),
     'fc2': nn.Linear(512, 256),
     #'drop1': nn.Dropout(.05),
     'out': nn.Linear(256, 10)})

In [8]:
criterion = TripletRegularizedCrossEntropyLoss(.01, .5)
sess = Session(model, criterion, optim.AdamW, 1e-3)

/home/drake/anaconda3/envs/.torch/lib/python3.7/site-packages/torch/cuda/__init__.py:135: UserWarning: 
    Found GPU0 GeForce GTX 770 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability that we support is 3.5.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


In [ ]:
lr_find(sess, trainloader, start_lr=1e-5)

In [9]:
sess.set_lr(1e-3)

In [10]:
validator = Validator(valloader, CustomOneHotAccuracy(), save_best=True, model_dir='./cifar10')
lr_scheduler = CosAnneal(len(trainloader), T_mult=2)
schedule = TrainingSchedule(trainloader, [lr_scheduler, validator])
sess.train(schedule, 128)

Validating: 100%|██████████| 313/313 [00:03<00:00, 92.68it/s]


Training Loss: 1.1112991571426392  Validaton Loss: 1.0888540744781494 Validation Accuracy: 0.6462


Validating: 100%|██████████| 313/313 [00:03<00:00, 91.85it/s]


Training Loss: 0.931346595287323  Validaton Loss: 0.8791949152946472 Validation Accuracy: 0.7091000000000001


Validating: 100%|██████████| 313/313 [00:03<00:00, 91.88it/s]


...

Training Loss: 0.05380900576710701  Validaton Loss: 0.6009554266929626 Validation Accuracy: 0.8718


Validating: 100%|██████████| 313/313 [00:03<00:00, 89.34it/s]

Training Loss: 0.05614975467324257  Validaton Loss: 0.6127774119377136 Validation Accuracy: 0.8711000000000001


Validating: 100%|██████████| 313/313 [00:03<00:00, 90.59it/s]

Training Loss: 0.26382341980934143  Validaton Loss: 0.6001372337341309 Validation Accuracy: 0.8328

